In [ ]:
from src.components.llm.custom.vllm.base_v2 import VllmServer
llm = VllmServer(api_url='http://103.145.79.20:6017/v1/completions', max_tokens=50)

In [1]:
from llama_index.llms.vllm import VllmServer
llm = VllmServer(api_url='http://103.145.79.20:6017/generate', max_new_tokens=50)

In [2]:
x = llm.stream_complete("việt nam là quốc gia ở đâu ?")

Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2.'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên của'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên của những'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên của những tổ'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên của những tổ chức'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là thành viên của những tổ chức quốc'
Beam candidate 0: 'việt nam là quốc gia ở đâu ?\n2. việt nam là 

ValidationError: 1 validation error for LLMCompletionEndEvent
response
  none is not an allowed value (type=type_error.none.not_allowed)

In [8]:
for i in x:
    print(i)

In [11]:
"""Example Python client for vllm.entrypoints.api_server"""

import argparse
import json
from typing import Iterable, List

import requests


def clear_line(n: int = 1) -> None:
    LINE_UP = '\033[1A'
    LINE_CLEAR = '\x1b[2K'
    for _ in range(n):
        print(LINE_UP, end=LINE_CLEAR, flush=True)


def post_http_request(prompt: str,
                      api_url: str,
                      n: int = 1,
                      stream: bool = False) -> requests.Response:
    headers = {"User-Agent": "Test Client"}
    pload = {
        "prompt": prompt,
        "n": n,
        "use_beam_search": True,
        "temperature": 0.0,
        "max_tokens": 16,
        "stream": stream,
    }
    response = requests.post(api_url, headers=headers, json=pload, stream=True)
    return response


def get_streaming_response(response: requests.Response) -> Iterable[List[str]]:
    for chunk in response.iter_lines(chunk_size=8192,
                                     decode_unicode=False,
                                     delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode("utf-8"))
            output = data["text"]
            yield output


def get_response(response: requests.Response) -> List[str]:
    data = json.loads(response.content)
    output = data["text"]
    return output


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--host", type=str, default="localhost")
    parser.add_argument("--port", type=int, default=8000)
    parser.add_argument("--n", type=int, default=4)
    parser.add_argument("--prompt", type=str, default="San Francisco is a")
    parser.add_argument("--stream", action="store_true")
    args = parser.parse_args()
    prompt = args.prompt
    api_url = f"http://{args.host}:{args.port}/generate"
    n = args.n
    stream = args.stream

    print(f"Prompt: {prompt!r}\n", flush=True)
    response = post_http_request(prompt, api_url, n, stream)

    if stream:
        num_printed_lines = 0
        for h in get_streaming_response(response):
            clear_line(num_printed_lines)
            num_printed_lines = 0
            for i, line in enumerate(h):
                num_printed_lines += 1
                print(f"Beam candidate {i}: {line!r}", flush=True)
    else:
        output = get_response(response)
        for i, line in enumerate(output):
            print(f"Beam candidate {i}: {line!r}", flush=True)

In [3]:
import weaviate
from src.components.vector_store.custom.weaviate.base import CustomWeaviateVectorStore
from src.components.embedding.custom.text_embeddings_inference.base import TextEmbeddingsInference
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex

client = weaviate.Client("http://localhost:9091")
import json

# auth_client_secret=weaviate.auth.AuthApiKey('jane-secret-key')
# embed_model = TextEmbeddingsInference(
#     base_url="http://localhost:8081",
#     timeout=60
# )

# DEVICE = "cuda:0"
# MODEL_NAME = "qducnguyen/vietnamese-bi-encoder"
# embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, 
#                                    max_length=256, device=DEVICE)

In [5]:
# Define the schema
class_schema = {
    "class": "Embedding",
    "properties": [
        {
            "name": "vector",
            "dataType": ["number[]"],
        }
    ]
}

# Create the schema if it doesn't exist
client.schema.create_class(class_schema)

In [5]:
from src.components.embedding.custom.text_embeddings_inference.base import (
    TextEmbeddingsInference,
)

embedding_model = TextEmbeddingsInference(
    base_url="http://34.29.222.41", timeout=60
)

In [6]:
from src.components.vector_store.custom.weaviate.base import CustomWeaviateVectorStore

DATA_COLLECTION = "Law07032024"

vector_store = CustomWeaviateVectorStore(weaviate_client=client, index_name=DATA_COLLECTION)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embedding_model,
)

In [11]:
from src.components.vector_store.custom.retriever import HybridRetrieverV2
vector_index_retriever = HybridRetrieverV2(
                index=index,
                similarity_top_k=20,
                retrieval_top_k=5,
                alpha=0.8
            )

text = "đi xe máy không đội mũ bảo hiểm bị phạt bao nhiêu tiền ?"
# text = ViTokenizer.tokenize(text.lower())


from src.components.rerank.custom.model_api.base import ModelAPIRerank 
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import MetadataMode, NodeWithScore, QueryBundle

node_postprocessors = [
    MetadataReplacementPostProcessor(target_metadata_key="window"),
    # SimilarityPostprocessor(
    #     similarity_cutoff=0.5
    # ),
] 

nodes = vector_index_retriever.retrieve(text)
# for postprocessor in node_postprocessors:
#     nodes = postprocessor.postprocess_nodes(
#         nodes, query_bundle=QueryBundle(text)
#     )

21:31:24.729 [INFO    ]                     httpx - HTTP Request: POST http://34.29.222.41/embed "HTTP/1.1 200 OK"


len(vector_nodes) 20
vector_nodes time 3.207819700241089
len(bm25_nodes) 20
bm25_nodes time 0.901313066482544
max_dense_score 0.5823841000000001
len(x) 5


In [12]:
nodes[0].metadata

{'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json',
 'source': '123/2021/NĐ-CP, Điều 2',
 'len_tokenizer': 16663,
 'parent_text': 'đường bộ; b) Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ.”; b) Điều khiển xe (kể cả rơ moóc và sơ mi rơ moóc) không gắn biển số (đối với loại xe có quy định phải gắn biển số); b) Người điều khiển xe ô tô, máy kéo và các loại xe tương tự xe ô tô không có hoặc không mang theo Giấy chứng nhận bảo hiểm trách nhiệm dân sự của chủ xe cơ giới còn hiệu lực. b) Thực hiện hành vi quy định tại khoản 2, khoản 3 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng; b) Điều khiển xe có Giấy phép lưu hành còn giá trị sử dụng nhưng tổng trọng lượng (khối lượng toàn bộ) của xe hoặc tải trọng trục xe (bao gồm cả hàng hóa xếp trên xe nếu có) vượt quá quy định trong Giấy phép lưu hành; b) Lưu hành phương tiện trên lãnh thổ Việt Nam quá thời hạn quy định dưới 30 ngày. b) Phạt tiền đ

In [9]:
from llama_index.core.response.notebook_utils import display_source_node
for n in vector_index_retriever.retrieve(text):
    display_source_node(n, source_length=1000, show_source_metadata=True)

17:29:58.812 [INFO    ]                     httpx - HTTP Request: POST http://34.29.222.41/embed "HTTP/1.1 200 OK"


vector_nodes time 1.8821566104888916
bm25_nodes time 1.1262292861938477
len(all_nodes) 32
18.79261 0.5823841000000001
17.551252 0.48049090000000005
20.899967 0.47861433
0.0 0.47400147000000004
18.638426 0.43821239999999995
0.0 0.43365549999999997
11.54544 0.42710983999999996
21.35604 0.42572600000000005
0.0 0.42505800000000005
0.0 0.42132879999999995
0.0 0.42132879999999995
0.0 0.41240895
0.0 0.41240895
0.0 0.40754193000000005
0.0 0.40754193000000005
18.70376 0.403609
0.0 0.40295590000000003
0.0 0.40259135
0.0 0.39863059999999995
17.940262 0.39816326
20.33075 0.0
17.632566 0.0
12.119919 0.0
11.647045 0.0
11.34596 0.0
11.247841 0.0
11.031069 0.0
11.004913 0.0
10.811857 0.0
10.613654 0.0
10.49155 0.0
10.406708 0.0


**Node ID:** 262d58bd-8581-4e7e-b331-969b78f8c040<br>**Similarity:** 18.79261<br>**Text:** b ) phạt tiền đến 15.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; b ) phạt tiền đến 75.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; 
 o ) chở người ngồi trên xe không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi , áp_giải người có hành_vi vi_phạm_pháp_luật . ” ; 
 c ) sửa_đổi khoản 5 điều 6 như sau : “ 5 . phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối_với người điều_khiển xe thực_hiện một trong các hành_vi vi_phạm sau đây : c ) điều_khiển xe ô_tô tải ( kể_cả rơ_moóc và sơ_mi rơ_moóc ) có kích_thước thùng_xe không đúng với thông_số kỹ_thuật được ghi trong giấy chứng_nhận kiểm_định an_toàn kỹ_thuật và bảo_vệ môi_trường của xe ; c ) thực_hiện_hành_vi quy_định tại điểm a khoản 3 điều này bị tịch_thu giấy_phép lưu_hành đã hết giá_trị sử_dụng hoặc không do...<br>**Metadata:** {'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json', 'source': '123/2021/NĐ-CP, Điều 2', 'len_tokenizer': 16663, 'parent_text': 'đường bộ; b) Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ.”; b) Điều khiển xe (kể cả rơ moóc và sơ mi rơ moóc) không gắn biển số (đối với loại xe có quy định phải gắn biển số); b) Người điều khiển xe ô tô, máy kéo và các loại xe tương tự xe ô tô không có hoặc không mang theo Giấy chứng nhận bảo hiểm trách nhiệm dân sự của chủ xe cơ giới còn hiệu lực. b) Thực hiện hành vi quy định tại khoản 2, khoản 3 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng; b) Điều khiển xe có Giấy phép lưu hành còn giá trị sử dụng nhưng tổng trọng lượng (khối lượng toàn bộ) của xe hoặc tải trọng trục xe (bao gồm cả hàng hóa xếp trên xe nếu có) vượt quá quy định trong Giấy phép lưu hành; b) Lưu hành phương tiện trên lãnh thổ Việt Nam quá thời hạn quy định dưới 30 ngày. b) Phạt tiền đến 15.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt; b) Phạt tiền đến 75.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt;\n\to) Chở người ngồi trên xe không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật.”;\n\tc) Sửa đổi khoản 5 Điều 6 như sau: “5. Phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: c) Điều khiển xe ô tô tải (kể cả rơ moóc và sơ mi rơ moóc) có kích thước thùng xe không đúng với thông số kỹ thuật được ghi trong Giấy chứng nhận kiểm định an toàn kỹ thuật và bảo vệ môi trường của xe; c) Thực hiện hành vi quy định tại điểm a khoản 3 Điều này bị tịch thu Giấy phép lưu hành đã hết giá trị sử dụng hoặc không do cơ quan có thẩm quyền cấp.', 'dense_score': 0.5823841000000001, 'bm25_score': 18.79261}<br>

**Node ID:** 0555f1f8-b7bb-40ad-8eea-61546090ae4c<br>**Similarity:** 17.551252<br>**Text:** điều 3 . kiên_quyết lập lại trật_tự kỷ_cương trong việc chấp_hành pháp_luật về trật_tự an_toàn giao_thông . bộ công_an tập_trung chỉ_đạo và tăng_cường_lực_lượng vào việc hướng_dẫn và điều_khiển giao_thông thông_suốt ; thực_hiện các biện_pháp cưỡng_chế thi_hành pháp_luật về trật_tự an_toàn giao_thông , trước_hết phải tập_trung kiểm_tra , xử_phạt nghiêm những người điều_khiển mô_tô không có giấy_phép lái_xe , kiên_quyết chấm_dứt tình_trạng người điều_khiển mô_tô không có giấy_phép lái_xe ; đồng_thời tăng_cường tuần_tra , kiểm_soát và xử_lý nghiêm_khắc những vi_phạm của người điều_khiển mô_tô , xe_máy như : chưa đủ tuổi quy_định , đua xe trái_phép , chạy quá tốc_độ , điều_khiển xe_máy lạng_lách , đánh_võng , ... . để hạn_chế thương_vong do chấn_thương sọ não từ tai_nạn giao_thông gây ra , cảnh_sát giao_thông phải kiên_quyết xử_phạt những người điều_khiển và ngồi trên mô_tô , xe_máy không đội mũ bảo_hiểm trên những đoạn đường bắt_buộc đội mũ bảo_hiểm . bộ giao_thông vận_tải có trách_nhi...<br>**Metadata:** {'filename': 'Nghị_quyết_số_13_2002_NQ-CP_về_các_giải_pháp_kiềm_.json', 'source': '13/2002/NQ-CP, Điều 3', 'len_tokenizer': 349, 'parent_text': 'Điều 3. Kiên quyết lập lại trật tự kỷ cương trong việc chấp hành pháp luật về trật tự an toàn giao thông. Bộ Công an tập trung chỉ đạo và tăng cường lực lượng vào việc hướng dẫn và điều khiển giao thông thông suốt; thực hiện các biện pháp cưỡng chế thi hành pháp luật về trật tự an toàn giao thông, trước hết phải tập trung kiểm tra, xử phạt nghiêm những người điều khiển mô tô không có giấy phép lái xe, kiên quyết chấm dứt tình trạng người điều khiển mô tô không có giấy phép lái xe; đồng thời tăng cường tuần tra, kiểm soát và xử lý nghiêm khắc những vi phạm của người điều khiển mô tô, xe máy như : chưa đủ tuổi quy định, đua xe trái phép, chạy quá tốc độ, điều khiển xe máy lạng lách, đánh võng,.... Để hạn chế thương vong do chấn thương sọ não từ tai nạn giao thông gây ra, cảnh sát giao thông phải kiên quyết xử phạt những người điều khiển và ngồi trên mô tô, xe máy không đội mũ bảo hiểm trên những đoạn đường bắt buộc đội mũ bảo hiểm. Bộ Giao thông vận tải có trách nhiệm rà soát, điều chỉnh lại những đoạn đường quy định phải đội mũ bảo hiểm khi đi mô tô, xe máy cho phù hợp. Bộ Giao thông vận tải phải thường xuyên kiểm tra các cơ sở đào tạo và công tác sát hạch cấp giấy phép lái xe, kiên quyết thu hồi giấy phép đào tạo của những cơ sở đào tạo lái xe không đủ tiêu chuẩn quy định, đình chỉ hoạt động và xử lý nghiêm khắc Hội đồng sát hạch nếu có sai phạm hoặc sát hạch viên có hành vi tiêu cực; ban hành quy định việc kiểm tra lại và đổi giấy phép lái xe theo lộ trình thích hợp, không gây phiền hà cho nhân dân. Bộ Công an chủ trì, phối hợp với Bộ Giao thông vận tải ban hành quy định việc đánh dấu số lần vi phạm Luật Giao thông đường bộ trên giấy phép lái xe hiện hành để quản lý người điều khiển phương tiện cơ giới đường bộ (kể cả người điều khiển mô tô), chỉ đạo việc cử cán bộ cảnh sát giao thông là sát hạch viên tham gia các Hội đồng sát hạch để cấp giấy phép lái xe của ngành giao thông', 'dense_score': 0.48049090000000005, 'bm25_score': 17.551252}<br>

**Node ID:** d9bd7510-ea9d-47ea-974d-5aef60387ffe<br>**Similarity:** 20.899967<br>**Text:** trên phần đường dành cho người đi bộ qua đường ; dừng xe nơi có biển “ cấm dừng xe và đỗ xe ” ; đỗ xe tại nơi có biển “ cấm đỗ xe ” hoặc biển “ cấm dừng xe và đỗ xe ” ; không tuân_thủ các quy_định về dừng xe , đỗ xe tại nơi đường_bộ giao nhau cùng mức với đường_sắt ; dừng xe , đỗ xe trong phạm_vi an_toàn của đường_sắt , trừ hành_vi vi_phạm_quy_định tại điểm b khoản 2 , điểm b khoản 3 điều 49 nghị_định này ; 
 i ) không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách khi điều_khiển xe tham_gia giao_thông trên đường_bộ ; 
 k ) chở người ngồi trên xe không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi ,<br>**Metadata:** {'filename': 'Nghị_định_100_2019_NĐ-CP_quy_định_về_xử_phạt_vi_ph.json', 'source': '100/2019/NĐ-CP, Chương II, Điều 6, Khoản 2', 'len_tokenizer': 476, 'parent_text': '2. Phạt tiền từ 200.000 đồng đến 300.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:\n\ta) Dừng xe, đỗ xe trên phần đường xe chạy ở đoạn đường ngoài đô thị nơi có lề đường;\n\tb) Không giảm tốc độ và nhường đường khi điều khiển xe chạy từ trong ngõ, đường nhánh ra đường chính;\n\tc) Điều khiển xe chạy quá tốc độ quy định từ 05 km/h đến dưới 10 km/h;\n\td) Điều khiển xe chạy tốc độ thấp mà không đi bên phải phần đường xe chạy gây cản trở giao thông;\n\tđ) Dừng xe, đỗ xe ở lòng đường đô thị gây cản trở giao thông; tụ tập từ 03 xe trở lên ở lòng đường, trong hầm đường bộ; đỗ, để xe ở lòng đường đô thị, hè phố trái quy định của pháp luật;\n\te) Không nhường đường cho xe xin vượt khi có đủ điều kiện an toàn; không nhường đường cho xe đi trên đường ưu tiên, đường chính từ bất kỳ hướng nào tới tại nơi đường giao nhau;\n\tg) Xe không được quyền ưu tiên lắp đặt, sử dụng thiết bị phát tín hiệu của xe được quyền ưu tiên;\n\th) Dừng xe, đỗ xe trên đường xe điện, điểm dừng đón trả khách của xe buýt, nơi đường bộ giao nhau, trên phần đường dành cho người đi bộ qua đường; dừng xe nơi có biển “Cấm dừng xe và đỗ xe”; đỗ xe tại nơi có biển “Cấm đỗ xe” hoặc biển “Cấm dừng xe và đỗ xe”; không tuân thủ các quy định về dừng xe, đỗ xe tại nơi đường bộ giao nhau cùng mức với đường sắt; dừng xe, đỗ xe trong phạm vi an toàn của đường sắt, trừ hành vi vi phạm quy định tại điểm b khoản 2, điểm b khoản 3 Điều 49 Nghị định này;\n\ti) Không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách khi điều khiển xe tham gia giao thông trên đường bộ;\n\tk) Chở người ngồi trên xe không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi,', 'dense_score': 0.47861433, 'bm25_score': 20.899967}<br>

**Node ID:** b5655282-ef24-4e86-9560-a64f924f918c<br>**Similarity:** 18.638426<br>**Text:** ô_tô . 6 . ngoài việc bị phạt tiền , người điều_khiển phương_tiện thực_hiện_hành_vi vi_phạm còn bị áp_dụng các hình_thức xử_phạt bổ_sung sau đây : 6 . ngoài việc bị áp_dụng hình_thức xử_phạt , người điều_khiển phương_tiện thực_hiện_hành_vi vi_phạm_quy_định tại điểm b , điểm c khoản 2 ; điểm a khoản 3 ; điểm b khoản 4 ; khoản 5 điều này còn bị áp_dụng biện_pháp khắc_phục hậu_quả : buộc tái_xuất phương_tiện khỏi việt_nam . ” . 6 . cục_trưởng cục cảnh_sát giao_thông , cục_trưởng cục cảnh_sát quản_lý hành_chính về trật_tự xã_hội , cục_trưởng cục cảnh_sát phòng cháy , chữa_cháy và cứu nạn , cứu_hộ , tư_lệnh cảnh_sát cơ_động có quyền : 6 . giám_đốc cảng_vụ hàng_hải , giám_đốc cảng_vụ hàng_không , giám_đốc cảng_vụ đường thủy_nội_địa có quyền : 
 e ) chở người ngồi trên xe_đạp máy ( kể_cả xe_đạp điện ) không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dướ...<br>**Metadata:** {'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json', 'source': '123/2021/NĐ-CP, Điều 2', 'len_tokenizer': 16663, 'parent_text': 'ô tô. 6. Ngoài việc bị phạt tiền, người điều khiển phương tiện thực hiện hành vi vi phạm còn bị áp dụng các hình thức xử phạt bổ sung sau đây: 6. Ngoài việc bị áp dụng hình thức xử phạt, người điều khiển phương tiện thực hiện hành vi vi phạm quy định tại điểm b, điểm c khoản 2; điểm a khoản 3; điểm b khoản 4; khoản 5 Điều này còn bị áp dụng biện pháp khắc phục hậu quả: Buộc tái xuất phương tiện khỏi Việt Nam.”. 6. Cục trưởng Cục Cảnh sát giao thông, Cục trưởng Cục Cảnh sát quản lý hành chính về trật tự xã hội, Cục trưởng Cục Cảnh sát phòng cháy, chữa cháy và cứu nạn, cứu hộ, Tư lệnh Cảnh sát cơ động có quyền: 6. Giám đốc Cảng vụ hàng hải, Giám đốc Cảng vụ hàng không, Giám đốc Cảng vụ đường thủy nội địa có quyền:\n\te) Chở người ngồi trên xe đạp máy (kể cả xe đạp điện) không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật.”.\n\ta) Điều khiển xe gắn biển số không đúng với Giấy đăng ký xe hoặc gắn biển số không do cơ quan có thẩm quyền cấp (kể cả rơ moóc và sơ mi rơ moóc); a) Thực hiện hành vi quy định tại khoản 1, khoản 3, điểm a khoản 4 Điều này còn bị tước quyền sử dụng Giấy phép lái xe (khi điều khiển ô tô, máy kéo và các loại xe tương tự ô tô), chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ (khi điều khiển xe máy chuyên dùng) từ 01 tháng đến 03 tháng; a) Phạt cảnh cáo; a) Phạt cảnh cáo;\n\tb) Sử dụng Giấy đăng ký xe, Giấy chứng nhận, tem kiểm định an toàn kỹ thuật và bảo vệ môi trường không do cơ quan có thẩm quyền cấp hoặc bị tẩy xóa; sử dụng Giấy đăng ký xe không đúng số khung, số máy của xe (kể cả rơ moóc và sơ mi rơ moóc); b)', 'dense_score': 0.43821239999999995, 'bm25_score': 18.638426}<br>

**Node ID:** 8ded3dbd-3de1-42e5-9713-e64e91303ca8<br>**Similarity:** 11.54544<br>**Text:** xe gắn_máy , các loại xe tương_tự xe mô_tô và các loại xe tương_tự xe gắn_máy không có hoặc không mang theo giấy chứng_nhận bảo_hiểm trách_nhiệm dân_sự của chủ xe cơ_giới còn hiệu_lực ; a ) điều_khiển phương_tiện không gắn ký_hiệu phân_biệt quốc_gia theo quy_định ; a ) phạt cảnh_cáo ; a ) phạt cảnh_cáo ; a ) phạt cảnh_cáo ; 
 b ) điều_khiển xe không có còi hoặc có nhưng còi không có tác_dụng ; b ) người điều_khiển xe mô_tô , xe gắn_máy , các loại xe tương_tự xe mô_tô và các loại xe tương_tự xe gắn_máy không mang theo giấy đăng_ký xe ; b ) điều_khiển phương_tiện không có giấy_phép vận_tải quốc_tế , phù_hiệu vận_tải quốc_tế liên_vận theo quy_định hoặc có nhưng đã hết giá_trị sử_dụng ; b ) phạt tiền đến 37.500.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; b ) phạt tiền đến 1.500.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ ,<br>**Metadata:** {'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json', 'source': '123/2021/NĐ-CP, Điều 2', 'len_tokenizer': 16663, 'parent_text': 'quan quản lý đường bộ ở khu vực thuộc Tổng cục Đường bộ Việt Nam, Trưởng đoàn thanh tra chuyên ngành của Sở Giao thông vận tải, Trưởng đoàn thanh tra chuyên ngành của Sở Tài nguyên và Môi trường, Trưởng đoàn thanh tra chuyên ngành của Tổng cục Môi trường, Trưởng đoàn thanh tra chuyên ngành của Tổng cục Đường bộ Việt Nam, Trưởng đoàn thanh tra chuyên ngành của Cục Đường sắt Việt Nam, Trưởng đoàn thanh tra chuyên ngành của cơ quan quản lý đường bộ ở khu vực thuộc Tổng cục Đường bộ Việt Nam có quyền:\n\ta) Điều khiển xe không có đủ đèn chiếu sáng, đèn soi biển số, đèn báo hãm, đèn tín hiệu, cần gạt nước, gương chiếu hậu, dây an toàn, dụng cụ thoát hiểm, thiết bị chữa cháy, đồng hồ báo áp lực hơi, đồng hồ báo tốc độ của xe hoặc có những thiết bị đó nhưng không có tác dụng, không đúng tiêu chuẩn thiết kế (đối với loại xe được quy định phải có những thiết bị đó), trừ các hành vi vi phạm quy định tại điểm m khoản 3 Điều 23, điểm q khoản 4 Điều 28 Nghị định này; a) Người điều khiển xe mô tô, xe gắn máy, các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy không có hoặc không mang theo Giấy chứng nhận bảo hiểm trách nhiệm dân sự của chủ xe cơ giới còn hiệu lực; a) Điều khiển phương tiện không gắn ký hiệu phân biệt quốc gia theo quy định; a) Phạt cảnh cáo; a) Phạt cảnh cáo; a) Phạt cảnh cáo;\n\tb) Điều khiển xe không có còi hoặc có nhưng còi không có tác dụng; b) Người điều khiển xe mô tô, xe gắn máy, các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy không mang theo Giấy đăng ký xe; b) Điều khiển phương tiện không có Giấy phép vận tải quốc tế, phù hiệu vận tải quốc tế liên vận theo quy định hoặc có nhưng đã hết giá trị sử dụng; b) Phạt tiền đến 37.500.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt; b) Phạt tiền đến 1.500.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ,', 'dense_score': 0.42710983999999996, 'bm25_score': 11.54544}<br>

In [10]:
from llama_index.llms.llama_cpp import LlamaCPP

ModuleNotFoundError: No module named 'llama_index.llms.llama_cpp'

In [ ]:
from src.components.llm.prompt_helper import get_prompt_style

prompt_style = get_prompt_style("mistral")

In [ ]:
prompt_style

In [ ]:
from llama_index.core.settings import Settings as LlamaIndexSettings

settings_kwargs = {
    "tfs_z": 1,  # ollama and llama-cpp
    "top_k": 40,  # ollama and llama-cpp
    "top_p": 0.9,  # ollama and llama-cpp
    "repeat_penalty": 1.1,  # ollama llama-cpp
    "n_gpu_layers": -1,
    "offload_kqv": True,
}
llm = LlamaCPP(
    model_path=str('models/ggml-vistral-7B-chat-q4_0.gguf'),
    temperature=0.1,
    max_new_tokens=512,
    context_window=3900,
    generate_kwargs={},
    callback_manager=LlamaIndexSettings.callback_manager,
    # All to GPU
    model_kwargs=settings_kwargs,
    # transform inputs into Llama2 format
    messages_to_prompt=prompt_style.messages_to_prompt,
    completion_to_prompt=prompt_style.completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from models/ggml-vistral-7B-chat-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32   

In [ ]:
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
    llm=llm
)

In [ ]:
vector_index_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    doc_ids=None,
    filters=(
        None
    )
)

In [ ]:
for n in vector_index_retriever.retrieve("Không đội mũ bảo hiểm phạt bao nhiêu tiền"):
    display_source_node(n, source_length=1000, show_source_metadata=True)

12:04:31.509 [INFO    ]                     httpx - HTTP Request: POST http://localhost:8081/embed "HTTP/1.1 200 OK"


**Node ID:** a80bb383-bde5-4045-b653-e5c64b7980ea<br>**Similarity:** 0.39883685<br>**Text:** điều 20 . xử_phạt đối_với hành_vi vi_phạm_quy_định về trích_lập quỹ bảo_vệ người được bảo_hiểm 
 1 . phạt cảnh_cáo đối_với hành_vi trích nộp quỹ bảo_vệ người được bảo_hiểm không đúng thời_hạn theo quy_định của pháp_luật . 
 2 . phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng đối_với hành_vi không trích nộp đủ quỹ bảo_vệ người được bảo_hiểm theo quy_định của pháp_luật . 
 3 . phạt tiền từ 60.000.000 đồng đến 70.000.000 đồng đối_với hành_vi vi_phạm không trích nộp quỹ bảo_vệ người được bảo_hiểm theo quy_định của pháp_luật .<br>**Metadata:** {'filename': 'Nghị_định_98_2013_NĐ-CP_quy_định_xử_phạt_vi_phạm_h.json', 'source': '98/2013/NĐ-CP, Chương 2, MỤC 3, Điều 20', 'len_tokenizer': 93, 'parent_text': 'Điều 20. Xử phạt đối với hành vi vi phạm quy định về trích lập Quỹ bảo vệ người được bảo hiểm\n1. Phạt cảnh cáo đối với hành vi trích nộp Quỹ bảo vệ người được bảo hiểm không đúng thời hạn theo quy định của pháp luật.\n2. Phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng đối với hành vi không trích nộp đủ Quỹ bảo vệ người được bảo hiểm theo quy định của pháp luật.\n3. Phạt tiền từ 60.000.000 đồng đến 70.000.000 đồng đối với hành vi vi phạm không trích nộp Quỹ bảo vệ người được bảo hiểm theo quy định của pháp luật.'}<br>

**Node ID:** 8ab15407-e7c6-4b15-8172-2efa8e2a10a7<br>**Similarity:** 0.38667929999999995<br>**Text:** b ) phạt tiền đến 15.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; b ) phạt tiền đến 75.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; 
 o ) chở người ngồi trên xe không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi , áp_giải người có hành_vi vi_phạm_pháp_luật . ” ; 
 c ) sửa_đổi khoản 5 điều 6 như sau : “ 5 . phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối_với người điều_khiển xe thực_hiện một trong các hành_vi vi_phạm sau đây : c ) điều_khiển xe ô_tô tải ( kể_cả rơ_moóc và sơ_mi rơ_moóc ) có kích_thước thùng_xe không đúng với thông_số kỹ_thuật được ghi trong giấy chứng_nhận kiểm_định an_toàn kỹ_thuật và bảo_vệ môi_trường của xe ; c ) thực_hiện_hành_vi quy_định tại điểm a khoản 3 điều này bị tịch_thu giấy_phép lưu_hành đã hết giá_trị sử_dụng hoặc không do...<br>**Metadata:** {'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json', 'source': '123/2021/NĐ-CP, Điều 2', 'len_tokenizer': 16663, 'parent_text': 'đường bộ; b) Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ.”; b) Điều khiển xe (kể cả rơ moóc và sơ mi rơ moóc) không gắn biển số (đối với loại xe có quy định phải gắn biển số); b) Người điều khiển xe ô tô, máy kéo và các loại xe tương tự xe ô tô không có hoặc không mang theo Giấy chứng nhận bảo hiểm trách nhiệm dân sự của chủ xe cơ giới còn hiệu lực. b) Thực hiện hành vi quy định tại khoản 2, khoản 3 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng; b) Điều khiển xe có Giấy phép lưu hành còn giá trị sử dụng nhưng tổng trọng lượng (khối lượng toàn bộ) của xe hoặc tải trọng trục xe (bao gồm cả hàng hóa xếp trên xe nếu có) vượt quá quy định trong Giấy phép lưu hành; b) Lưu hành phương tiện trên lãnh thổ Việt Nam quá thời hạn quy định dưới 30 ngày. b) Phạt tiền đến 15.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt; b) Phạt tiền đến 75.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt;\n\to) Chở người ngồi trên xe không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật.”;\n\tc) Sửa đổi khoản 5 Điều 6 như sau: “5. Phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: c) Điều khiển xe ô tô tải (kể cả rơ moóc và sơ mi rơ moóc) có kích thước thùng xe không đúng với thông số kỹ thuật được ghi trong Giấy chứng nhận kiểm định an toàn kỹ thuật và bảo vệ môi trường của xe; c) Thực hiện hành vi quy định tại điểm a khoản 3 Điều này bị tịch thu Giấy phép lưu hành đã hết giá trị sử dụng hoặc không do cơ quan có thẩm quyền cấp.'}<br>

**Node ID:** 210346c0-0861-408d-8ba9-ed80dbe610a5<br>**Similarity:** 0.38451029999999997<br>**Text:** 1 . phạt tiền từ 500.000 đồng đến 1.000.000 đồng đối_với hành_vi không trang_bị hoặc trang_bị không đủ áo_phao hoặc dụng_cụ nổi cứu_sinh cá_nhân theo quy_định , mức phạt tính trên mỗi áo_phao , dụng_cụ nổi cứu_sinh cá_nhân .<br>**Metadata:** {'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json', 'source': '139/2021/ND-CP, Chương II, Mục 2, Điều 16, Khoản 1', 'len_tokenizer': 41, 'parent_text': '1. Phạt tiền từ 500.000 đồng đến 1.000.000 đồng đối với hành vi không trang bị hoặc trang bị không đủ áo phao hoặc dụng cụ nổi cứu sinh cá nhân theo quy định, mức phạt tính trên mỗi áo phao, dụng cụ nổi cứu sinh cá nhân.'}<br>

**Node ID:** f7716b29-1070-44e7-8456-255629f1cc6e<br>**Similarity:** 0.38451029999999997<br>**Text:** 1 . phạt tiền từ 500.000 đồng đến 1.000.000 đồng đối_với hành_vi không trang_bị hoặc trang_bị không đủ áo_phao hoặc dụng_cụ nổi cứu_sinh cá_nhân theo quy_định , mức phạt tính trên mỗi áo_phao , dụng_cụ nổi cứu_sinh cá_nhân .<br>**Metadata:** {'filename': 'Nghị_định_139_2021_NĐ-CP_quy_định_về_xử_phạt_vi_ph.json', 'source': '139/2021/NĐ-CP, Chương II, Mục 2, Điều 16, Khoản 1', 'len_tokenizer': 41, 'parent_text': '1. Phạt tiền từ 500.000 đồng đến 1.000.000 đồng đối với hành vi không trang bị hoặc trang bị không đủ áo phao hoặc dụng cụ nổi cứu sinh cá nhân theo quy định, mức phạt tính trên mỗi áo phao, dụng cụ nổi cứu sinh cá nhân.'}<br>

...

**Node ID:** bfe015c2-6f90-471b-8a77-57a7ddd45d8a<br>**Similarity:** 0.3391267<br>**Text:** điều 35 . vi_phạm_quy_định về không bảo_đảm an_toàn dẫn đến tai_nạn , thương_tích cho trẻ_em 
 1 . phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng đối_với một trong các hành_vi sau đây : 
 a ) không thông_báo , cảnh_báo những nơi nguy_hiểm , độc_hại , dễ cháy , nổ , điện giật , rơi , ngã và các nguy_cơ khác dẫn đến tai_nạn , thương_tích cho trẻ_em ; 
 b ) không tuân_thủ đầy_đủ , kịp_thời các biện_pháp_bảo_đảm an_toàn về sản_xuất , kinh_doanh , lưu_hành các phương_tiện giao_thông vận_tải dẫn đến tai_nạn , thương_tích cho trẻ_em . 
 2 . biện_pháp khắc_phục hậu_quả : 
 a ) buộc thực_hiện kịp_thời các biện_pháp để khắc_phục tình_trạng thiếu an_toàn đối_với hành_vi vi_phạm tại khoản 1 điều này ; 
 b ) buộc chịu mọi chi_phí khám bệnh , chữa bệnh ( nếu có ) cho trẻ_em đối_với hành_vi vi_phạm tại khoản 1 điều này .<br>**Metadata:** {'filename': 'Nghị_định_130_2021_NĐ-CP_quy_định_về_xử_phạt_vi_ph.json', 'source': '130/2021/NĐ-CP, Chương II, Mục 2, Điều 35', 'len_tokenizer': 155, 'parent_text': 'Điều 35. Vi phạm quy định về không bảo đảm an toàn dẫn đến tai nạn, thương tích cho trẻ em\n1. Phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng đối với một trong các hành vi sau đây:\n\ta) Không thông báo, cảnh báo những nơi nguy hiểm, độc hại, dễ cháy, nổ, điện giật, rơi, ngã và các nguy cơ khác dẫn đến tai nạn, thương tích cho trẻ em;\n\tb) Không tuân thủ đầy đủ, kịp thời các biện pháp bảo đảm an toàn về sản xuất, kinh doanh, lưu hành các phương tiện giao thông vận tải dẫn đến tai nạn, thương tích cho trẻ em.\n2. Biện pháp khắc phục hậu quả:\n\ta) Buộc thực hiện kịp thời các biện pháp để khắc phục tình trạng thiếu an toàn đối với hành vi vi phạm tại khoản 1 Điều này;\n\tb) Buộc chịu mọi chi phí khám bệnh, chữa bệnh (nếu có) cho trẻ em đối với hành vi vi phạm tại khoản 1 Điều này.'}<br>

**Node ID:** 742e22a6-fb95-42e9-b21a-c0e1a5d968d8<br>**Similarity:** 0.33852530000000003<br>**Text:** điều 6 . xử_phạt người điều_khiển , người ngồi trên xe mô_tô , xe gắn_máy ( kể_cả xe_máy_điện ) , các loại xe tương_tự xe mô_tô và các loại xe tương_tự xe gắn_máy vi_phạm_quy_tắc giao_thông đường_bộ 
 1 . phạt tiền từ 60.000 đồng đến 80.000 đồng đối_với một trong các hành_vi vi_phạm sau đây : 
 a ) không chấp_hành hiệu_lệnh , chỉ_dẫn của biển_báo_hiệu , vạch kẻ đường , trừ các hành_vi vi_phạm_quy_định tại điểm a , điểm d , điểm đ , điểm h khoản 2 ; điểm c , điểm đ , điểm h , điểm m khoản 3 ; điểm c , điểm d , điểm g , điểm i khoản 4 ; điểm a , điểm b , điểm c , điểm d , điểm e khoản 5 ; điểm b khoản 7 ; điểm a khoản 8 ; điểm d khoản 9 điều này ; 
 b ) không có báo_hiệu xin vượt trước khi vượt ; 
 c ) không giữ khoảng_cách an_toàn để xảy ra va_chạm với xe chạy liền trước hoặc không giữ khoảng_cách theo quy_định của biển_báo_hiệu “ cự_ly tối_thiểu giữa hai xe ” ; 
 d ) chuyển_hướng không nhường quyền đi trước cho : người đi bộ , xe_lăn của người khuyết_tật qua đường tại nơi có vạch kẻ...<br>**Metadata:** {'filename': 'Nghị_định_46_2016_NĐ-CP_quy_định_xử_phạt_vi_phạm_h.json', 'source': '46/2016/NĐ-CP, Chương II, Điều 6', 'len_tokenizer': 2440, 'parent_text': 'Điều 6. Xử phạt người điều khiển, người ngồi trên xe mô tô, xe gắn máy (kể cả xe máy điện), các loại xe tương tự xe mô tô và các loại xe tương tự xe gắn máy vi phạm quy tắc giao thông đường bộ\n1. Phạt tiền từ 60.000 đồng đến 80.000 đồng đối với một trong các hành vi vi phạm sau đây:\n\ta) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại Điểm a, Điểm d, Điểm đ, Điểm h Khoản 2; Điểm c, Điểm đ, Điểm h, Điểm m Khoản 3; Điểm c, Điểm d, Điểm g, Điểm i Khoản 4; Điểm a, Điểm b, Điểm c, Điểm d, Điểm e Khoản 5; Điểm b Khoản 7; Điểm a Khoản 8; Điểm d Khoản 9 Điều này;\n\tb) Không có báo hiệu xin vượt trước khi vượt;\n\tc) Không giữ khoảng cách an toàn để xảy ra va chạm với xe chạy liền trước hoặc không giữ khoảng cách theo quy định của biển báo hiệu “Cự ly tối thiểu giữa hai xe”;\n\td) Chuyển hướng không nhường quyền đi trước cho: Người đi bộ, xe lăn của người khuyết tật qua đường tại nơi có vạch kẻ đường dành cho người đi bộ; xe thô sơ đang đi trên phần đường dành cho xe thô sơ;\n\tđ) Chuyển hướng không nhường đường cho: Các xe đi ngược chiều; người đi bộ, xe lăn của người khuyết tật đang qua đường tại nơi không có vạch kẻ đường cho người đi bộ;\n\te) Lùi xe mô tô ba bánh không quan sát hoặc không có tín hiệu báo trước;\n\tg) Sử dụng đèn chiếu xa khi tránh xe đi ngược chiều;\n\th) Chở người ngồi trên xe sử dụng ô (dù);\n\ti) Không tuân thủ các quy định về nhường đường tại nơi đường giao nhau, trừ các hành vi vi phạm quy định tại các Điểm d Khoản 2, Điểm b Khoản 3 Điều này.\n2. Phạt tiền từ 80.000 đồng đến 100.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:\n\ta) Chuyển làn đường không đúng nơi được phép hoặc không có tín hiệu báo trước;\n\tb) Điều khiển xe chạy dàn hàng ngang từ 3 (ba) xe trở lên;\n\tc) Không sử dụng đèn chiếu sáng trong thời gian từ 19 giờ ngày'}<br>

**Node ID:** 4d0b254b-b192-44b8-897e-3cb76445b2ee<br>**Similarity:** 0.32410866000000005<br>**Text:** 6 . phạt tiền từ 40.000.000 đồng ( bốn mươi triệu đồng ) đến 60.000.000 đồng ( sáu mươi triệu đồng ) đối_với hành_vi không mua bảo_hiểm trách_nhiệm dân_sự hoặc không thực_hiện các biện_pháp_bảo_đảm khác đối_với hành_khách , hành_lý , hàng_hóa , bưu gửi .<br>**Metadata:** {'filename': 'Nghị_định_162_2018_NĐ-CP_quy_định_về_xử_phạt_vi_ph.json', 'source': '162/2018/NĐ-CP, Chương II, Mục 6, Điều 24, Khoản 6', 'len_tokenizer': 55, 'parent_text': '6. Phạt tiền từ 40.000.000 đồng (bốn mươi triệu đồng) đến 60.000.000 đồng (sáu mươi triệu đồng) đối với hành vi không mua bảo hiểm trách nhiệm dân sự hoặc không thực hiện các biện pháp bảo đảm khác đối với hành khách, hành lý, hàng hóa, bưu gửi.'}<br>

**Node ID:** db896939-efdd-4fdc-ad63-5090d702a1d5<br>**Similarity:** 0.3227502<br>**Text:** 5 . phạt tiền từ 800.000 đồng đến 1.000.000 đồng đối_với một trong các hành_vi vi_phạm sau đây : 
 a ) chạy xe trong hầm đường_bộ không sử_dụng đèn_chiếu sáng ; lùi xe , quay đầu xe trong hầm đường_bộ ; 
 b ) không thực_hiện biện_pháp_bảo_đảm an_toàn theo quy_định khi phương_tiện bị hư_hỏng trên đoạn đường_bộ giao nhau cùng mức với đường_sắt ; 
 c ) dừng xe , đỗ xe trong hầm đường_bộ không đúng nơi quy_định .<br>**Metadata:** {'filename': 'Nghị_định_46_2016_NĐ-CP_quy_định_xử_phạt_vi_phạm_h.json', 'source': '46/2016/NĐ-CP, Chương II, Điều 7, Khoản 5', 'len_tokenizer': 82, 'parent_text': '5. Phạt tiền từ 800.000 đồng đến 1.000.000 đồng đối với một trong các hành vi vi phạm sau đây:\n\ta) Chạy xe trong hầm đường bộ không sử dụng đèn chiếu sáng; lùi xe, quay đầu xe trong hầm đường bộ;\n\tb) Không thực hiện biện pháp bảo đảm an toàn theo quy định khi phương tiện bị hư hỏng trên đoạn đường bộ giao nhau cùng mức với đường sắt;\n\tc) Dừng xe, đỗ xe trong hầm đường bộ không đúng nơi quy định.'}<br>

**Node ID:** 7e1ec46a-25f2-43f2-8009-9e2c6db6e345<br>**Similarity:** 0.32269585<br>**Text:** trên phần đường dành cho người đi bộ qua đường ; dừng xe nơi có biển “ cấm dừng xe và đỗ xe ” ; đỗ xe tại nơi có biển “ cấm đỗ xe ” hoặc biển “ cấm dừng xe và đỗ xe ” ; không tuân_thủ các quy_định về dừng xe , đỗ xe tại nơi đường_bộ giao nhau cùng mức với đường_sắt ; dừng xe , đỗ xe trong phạm_vi an_toàn của đường_sắt , trừ hành_vi vi_phạm_quy_định tại điểm b khoản 2 , điểm b khoản 3 điều 49 nghị_định này ; 
 i ) không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách khi điều_khiển xe tham_gia giao_thông trên đường_bộ ; 
 k ) chở người ngồi trên xe không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi ,<br>**Metadata:** {'filename': 'Nghị_định_100_2019_NĐ-CP_quy_định_về_xử_phạt_vi_ph.json', 'source': '100/2019/NĐ-CP, Chương II, Điều 6, Khoản 2', 'len_tokenizer': 476, 'parent_text': '2. Phạt tiền từ 200.000 đồng đến 300.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:\n\ta) Dừng xe, đỗ xe trên phần đường xe chạy ở đoạn đường ngoài đô thị nơi có lề đường;\n\tb) Không giảm tốc độ và nhường đường khi điều khiển xe chạy từ trong ngõ, đường nhánh ra đường chính;\n\tc) Điều khiển xe chạy quá tốc độ quy định từ 05 km/h đến dưới 10 km/h;\n\td) Điều khiển xe chạy tốc độ thấp mà không đi bên phải phần đường xe chạy gây cản trở giao thông;\n\tđ) Dừng xe, đỗ xe ở lòng đường đô thị gây cản trở giao thông; tụ tập từ 03 xe trở lên ở lòng đường, trong hầm đường bộ; đỗ, để xe ở lòng đường đô thị, hè phố trái quy định của pháp luật;\n\te) Không nhường đường cho xe xin vượt khi có đủ điều kiện an toàn; không nhường đường cho xe đi trên đường ưu tiên, đường chính từ bất kỳ hướng nào tới tại nơi đường giao nhau;\n\tg) Xe không được quyền ưu tiên lắp đặt, sử dụng thiết bị phát tín hiệu của xe được quyền ưu tiên;\n\th) Dừng xe, đỗ xe trên đường xe điện, điểm dừng đón trả khách của xe buýt, nơi đường bộ giao nhau, trên phần đường dành cho người đi bộ qua đường; dừng xe nơi có biển “Cấm dừng xe và đỗ xe”; đỗ xe tại nơi có biển “Cấm đỗ xe” hoặc biển “Cấm dừng xe và đỗ xe”; không tuân thủ các quy định về dừng xe, đỗ xe tại nơi đường bộ giao nhau cùng mức với đường sắt; dừng xe, đỗ xe trong phạm vi an toàn của đường sắt, trừ hành vi vi phạm quy định tại điểm b khoản 2, điểm b khoản 3 Điều 49 Nghị định này;\n\ti) Không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách khi điều khiển xe tham gia giao thông trên đường bộ;\n\tk) Chở người ngồi trên xe không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi,'}<br>

In [ ]:
import requests

url = 'http://localhost:8082/rerank'

query_and_nodes = {}
query_and_nodes["query"] = "what is dog"
query_and_nodes["raw_scores"] = False
# query_and_nodes["return_text"] = False
# query_and_nodes["truncate"] = False
query_and_nodes["texts"] = [
    "dog", "cat"
]
headers = {
    'Content-Type': 'application/json'
}

response = requests.post(url=url, json=query_and_nodes, headers=headers)
print(response.json())

{'error': 'Backend error: MatMulUnexpectedStriding { lhs_l: Layout { shape: [2, 1024], stride: [8192, 1], start_offset: 0 }, rhs_l: Layout { shape: [1024, 1024], stride: [1, 1024], start_offset: 0 }, bmnk: (1, 2, 1024, 1024), msg: "non-contiguous lhs" }', 'error_type': 'Backend'}


In [ ]:
query_and_nodes

{'query': 'What is Deep learning',
 'raw_scores': False,
 'texts': ['deep', 'learning']}

In [ ]:
i = 1
while i < 10:
    print(1)
    i += 1
    if i == 2:
        break

1
